

### <span style="color:purple; font-weight:bold;">Project Name: Netflix Content Strategy Analysis</span>
### <span style="color:purple; font-weight:bold;">Person Name: Anirban Bose</span>
<h3><strong style="color: purple;">Data Source: </strong><a href="https://statso.io/netflix-content-strategy-case-study/" target="_blank">Netflix Content Strategy Case Study</a></h3>


#### **I. Import the modules and packages**

In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 
import plotly.io as pio
from IPython.display import display, HTML

In [2]:
# Set the default template to Dark theme
pio.templates.default = "plotly_dark"

#### **II. Declare the global variables, dictionaries etc.**

In [3]:
# Map month numbers to month names
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}

#### **III. Functions**

In [4]:
# Function to convert column names to snakecase

def func_convert_to_snake_case(df):
    """
    Converts column names of a pandas DataFrame to snake_case.
    
    Parameters:
        df (pd.DataFrame): The DataFrame whose columns need to be converted.
        
    Returns:
        pd.DataFrame: The DataFrame with updated column names in snake_case.
    """
    df.columns = (
        df.columns.str.strip()  # Remove leading/trailing spaces
        .str.lower()  # Convert to lowercase
        .str.replace(r"[ ()-]", "_", regex=True)  # Replace spaces, parentheses, and hyphens with underscores
        .str.replace(r"__+", "_", regex=True)  # Replace multiple underscores with a single underscore
        .str.rstrip("_")  # Remove trailing underscores
        .str.replace("?", "", regex=False) # Remove question mark from column name
    )
    return df

In [5]:
# Function to convert datatypes
def func_change_dtypes(df):
    """
    Cleans and converts specific columns of a DataFrame:
    - Converts 'available_globally' to boolean.
    - Converts 'release_date' to datetime.
    - Converts 'hours_viewed' to numeric.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame to clean and transform.
    
    Returns:
        pd.DataFrame: The cleaned and transformed DataFrame.
    """
    # Step 1: Convert 'available_globally' to boolean
    df["available_globally"] = df["available_globally"].map({"Yes": True, "No": False})
    
    # Step 2: Convert 'release_date' to datetime
    df["release_date"] = pd.to_datetime(df["release_date"], format="%Y-%m-%d", errors="coerce")
    
    # Step 3: Convert 'hours_viewed' to numeric
    df["hours_viewed"] = (
            df["hours_viewed"]
            .str.replace(",", "", regex=False)  # Remove commas
            .astype(int)  # Convert to integer
        )
    
    return df

In [6]:
# Function for feature engineering

def func_feature_engineering(df, date_col):

    # Extract the month from the date column
    df["release_month"] = df[date_col].dt.month

    df["release_month_name"] = df["release_month"].map(month_map)    

    return df

In [7]:
# Common function for bar chart
def create_bar_chart(df, group_col, title, x_label, y_label, group_category):
    """
    Generates a bar chart for `hours_viewed` grouped by the specified column.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        group_col (str): The column name to group by.
        title (str): The title of the chart.
        x_label (str): Label for the x-axis.
        y_label (str): Label for the y-axis.
    
    Returns:
        None: Displays the chart.
    """
     # Group and sort data
    if group_category == "sum":
        grouped_data = (
            df[[group_col, 'hours_viewed']]
            .groupby(group_col, as_index=False)
            .sum()
            .sort_values(by="hours_viewed", ascending=False)  # Sort in descending order
        )
    elif (group_category == "mean" or group_category == "avg"):
        grouped_data = (
            df[[group_col, 'hours_viewed']]
            .groupby(group_col, as_index=False)
            .mean()
            .sort_values(by="hours_viewed", ascending=False)  # Sort in descending order
        )

    fig = px.bar(
        grouped_data,
        x=group_col,
        y="hours_viewed",
        title=title,
        labels={group_col: x_label, "hours_viewed": y_label},
        color=group_col
    )
    fig.show()

In [8]:
def func_plot_monthly_viewership(df, month_col, viewership_col, title):
    
    # Aggregate viewership hours by release month
    monthly_viewership = df.groupby(month_col)[viewership_col].sum()

    # Create the line + markers plot
    fig = go.Figure(data=[
        go.Scatter(
            x=monthly_viewership.index,
            y=monthly_viewership.values,
            mode='lines+markers',
            marker=dict(color='blue'),
            line=dict(color='blue')
        )
    ])

    # Update layout with a custom title and axis labels
    fig.update_layout(
        title=title,
        xaxis_title='Month',
        yaxis_title='Total Hours Viewed (in billions)',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        height=600,
        width=1000
    )

    # Show the plot
    fig.show()

Read the data

In [9]:
# Go back to parent directory
parent_dir = os.path.dirname(os.getcwd())

# Create the filepath
filename = 'netflix_content_2023.csv'
filename_folder = 'input'
filepath = os.path.join(parent_dir, filename_folder, filename)

# Read the csv file and create the dataframe
df_netflix_content = pd.read_csv(filepath)

In [10]:
# Get the number of rows and columns
df_netflix_content.shape

(24812, 6)

Basic statistical analysis and data cleaning

In [11]:
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                24812 non-null  object
 1   Available Globally?  24812 non-null  object
 2   Release Date         8166 non-null   object
 3   Hours Viewed         24812 non-null  object
 4   Language Indicator   24812 non-null  object
 5   Content Type         24812 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


We realize here that the the columns are not in standard format and we need to convert them to snakecase

In [12]:
# convert function to snakecase
df_netflix_content = func_convert_to_snake_case(df_netflix_content)
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               24812 non-null  object
 1   available_globally  24812 non-null  object
 2   release_date        8166 non-null   object
 3   hours_viewed        24812 non-null  object
 4   language_indicator  24812 non-null  object
 5   content_type        24812 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


In [13]:
# check if available_globally column is a binary. If yes, then this column can be converted to boolean
for col in df_netflix_content.columns:
    unique_values = df_netflix_content[col].unique()
    print(f"{col}: {unique_values}")

title: ['The Night Agent: Season 1' 'Ginny & Georgia: Season 2'
 'The Glory: Season 1 // 더 글로리: 시즌 1' ... 'Terror y Feria: Season 1'
 'The Witcher: A Look Inside the Episodes: Limited Series'
 'We Are Black and British: Season 1']
available_globally: ['Yes' 'No']
release_date: ['2023-03-23' '2023-01-05' '2022-12-30' ... '2023-12-04' '2023-12-21'
 '2015-12-04']
hours_viewed: ['81,21,00,000' '66,51,00,000' '62,28,00,000' '50,77,00,000'
 '50,30,00,000' '44,06,00,000' '42,96,00,000' '40,25,00,000'
 '30,21,00,000' '26,62,00,000' '26,26,00,000' '25,25,00,000'
 '25,15,00,000' '24,99,00,000' '23,50,00,000' '23,48,00,000'
 '22,97,00,000' '22,11,00,000' '21,41,00,000' '20,97,00,000'
 '20,65,00,000' '20,55,00,000' '20,18,00,000' '20,07,00,000'
 '19,47,00,000' '19,29,00,000' '18,40,00,000' '18,23,00,000'
 '18,18,00,000' '17,68,00,000' '17,55,00,000' '17,43,00,000'
 '17,36,00,000' '17,24,00,000' '17,01,00,000' '16,83,00,000'
 '16,30,00,000' '16,20,00,000' '16,11,00,000' '15,76,00,000'
 '15,53,00,00

We realize that the datatypes of the following columns need to be change:
1. available_globally: from string to boolean
2. release_date: from string to date
3. hours_viewed: from string to integer

In [14]:
df_netflix_content = func_change_dtypes(df_netflix_content)
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               24812 non-null  object        
 1   available_globally  24812 non-null  bool          
 2   release_date        8166 non-null   datetime64[ns]
 3   hours_viewed        24812 non-null  int32         
 4   language_indicator  24812 non-null  object        
 5   content_type        24812 non-null  object        
dtypes: bool(1), datetime64[ns](1), int32(1), object(3)
memory usage: 896.7+ KB


Feature Engineering

1. We create a column release_month to analyze monthly trends

In [15]:
df_netflix_content = func_feature_engineering(df_netflix_content, 'release_date')
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               24812 non-null  object        
 1   available_globally  24812 non-null  bool          
 2   release_date        8166 non-null   datetime64[ns]
 3   hours_viewed        24812 non-null  int32         
 4   language_indicator  24812 non-null  object        
 5   content_type        24812 non-null  object        
 6   release_month       8166 non-null   float64       
 7   release_month_name  8166 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(1), int32(1), object(4)
memory usage: 1.3+ MB


In [16]:
df_netflix_content.head(5)

,title,available_globally,release_date,hours_viewed,language_indicator,content_type,release_month,release_month_name
0,The Night Agent: Season 1,True,2023-03-23,812100000,English,Show,3.0,March
1,Ginny & Georgia: Season 2,True,2023-01-05,665100000,English,Show,1.0,January
2,The Glory: Season 1 // 더 글로리: 시즌 1,True,2022-12-30,622800000,Korean,Show,12.0,December
3,Wednesday: Season 1,True,2022-11-23,507700000,English,Show,11.0,November
4,Queen Charlotte: A Bridgerton Story,True,2023-05-04,503000000,English,Movie,5.0,May


Exploratory Data Analysis (EDA)

In [17]:
# List of configurations for the bar charts
chart_configs = [
    {
        "group_col": "available_globally",
        "title": "Total Hours Viewed by Available Globally",
        "x_label": "Available Globally",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "available_globally",
        "title": "Average Hours Viewed by Available Globally",
        "x_label": "Available Globally",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    },
    {
        "group_col": "language_indicator",
        "title": "Total Hours Viewed by Language Indicator",
        "x_label": "Language Indicator",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "language_indicator",
        "title": "Average Hours Viewed by Language Indicator",
        "x_label": "Language Indicator",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    },
    {
        "group_col": "content_type",
        "title": "Total Hours Viewed by Content Type",
        "x_label": "Content Type",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "content_type",
        "title": "Average Hours Viewed by Content Type",
        "x_label": "Content Type",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    }
]

# Loop through the configurations and create bar charts
for config in chart_configs:
    create_bar_chart(
        df=df_netflix_content,
        group_col=config["group_col"],
        title=config["title"],
        x_label=config["x_label"],
        y_label=config["y_label"],
        group_category=config["group_category"]
    )

Insights:

1. Clearly, global shows are more popular and have more hours of viewership.
2. Curiously, even though English language has more hours of viewership, Korean language has more average viewership
3. Overall, shows are more popular than movies.

In [18]:
func_plot_monthly_viewership(
    df=df_netflix_content,
    month_col='release_month',
    viewership_col='hours_viewed',
    title='Total Viewership Hours by Release Month (2023)'
)

Insights

1. June and Dec seems to have noticiable spikes. Is it because of the summer and winter holidays that people have more time to watch shows?
2. Let us see what they are watching

In [19]:
# Create a categorical column for sorting by natural month order
df_netflix_content["release_month_name"] = pd.Categorical(
    df_netflix_content["release_month_name"],
    categories=[
        "January", "February", "March", "April", "May", "June", 
        "July", "August", "September", "October", "November", "December"
    ],
    ordered=True
)

# Aggregate data
table = (
    df_netflix_content.groupby(["release_month_name", "language_indicator", "content_type"])
    .agg(
        total_hours_viewed=("hours_viewed", "sum"),
        average_hours_viewed=("hours_viewed", "mean"),
        total_items=("title", "size"),
    )
    .reset_index()
    .sort_values(by=["release_month_name", "average_hours_viewed"],ascending=[True, False])  
    .reset_index(drop=True)
)

# Format total and average hours with comma separators
table["total_hours_viewed"] = table["total_hours_viewed"].apply(lambda x: f"{x:,.0f}")
table["average_hours_viewed"] = table["average_hours_viewed"].apply(lambda x: f"{x:,.0f}")


In [20]:
table

,release_month_name,language_indicator,content_type,total_hours_viewed,average_hours_viewed,total_items
0,January,Korean,Movie,"318,000,000","53,000,000",6
1,January,Korean,Show,"561,000,000","33,000,000",17
2,January,English,Show,"3,963,100,000","13,479,932",294
3,January,Non-English,Show,"231,400,000","9,256,000",25
4,January,English,Movie,"1,671,100,000","9,232,597",181
...,...,...,...,...,...,...
139,December,Japanese,Movie,"46,100,000","3,546,154",13
140,December,Hindi,Movie,"1,900,000","1,900,000",1
141,December,Hindi,Show,"8,100,000","1,350,000",6
142,December,Russian,Movie,0,nan,0


In [21]:
# Convert the table to an HTML table with scrollable styling
# Display the table in IPython
display(HTML(f"""
<div style="overflow-x:auto; max-height:400px; border:1px solid #ccc; padding:10px;">
    {table.to_html(index=False, escape=False)}
</div>
"""))

release_month_name,language_indicator,content_type,total_hours_viewed,average_hours_viewed,total_items
January,Korean,Movie,"318,000,000","53,000,000",6
January,Korean,Show,"561,000,000","33,000,000",17
January,English,Show,"3,963,100,000","13,479,932",294
January,Non-English,Show,"231,400,000","9,256,000",25
January,English,Movie,"1,671,100,000","9,232,597",181
January,Japanese,Movie,"107,200,000","8,933,333",12
January,Japanese,Show,"229,000,000","8,807,692",26
January,Hindi,Movie,"48,100,000","6,871,429",7
January,Non-English,Movie,"131,500,000","3,652,778",36
January,Hindi,Show,"11,200,000","2,800,000",4


In [1]:
# ABC